In [1]:
## https://www.tensorflow.org/get_started/mnist/pros

In [2]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [3]:
import tensorflow as tf
sess = tf.InteractiveSession()

## Softmax Regression Model

In [8]:
# pixel 28x28=784
# 10 output states - numbers 1-10
x = tf.placeholder(tf.float32, shape=[None, 784])
y_ = tf.placeholder(tf.float32, shape=[None, 10])

In [9]:
# weights and biases
W = tf.Variable(tf.zeros([784,10]))
b = tf.Variable(tf.zeros([10]))
sess.run(tf.global_variables_initializer())

In [28]:
# vectorized model
y = tf.matmul(x,W) + b
tf.Graph()

In [16]:
#loss function  - internally calls tf.nn.softmax_cross_entropy_with_logits and tf.reduce_mean 

cross_entropy = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=y_, logits=y,dim=1))


## Train the model

In [17]:
train_step = tf.train.GradientDescentOptimizer(0.5).minimize(cross_entropy)


In [25]:
# feed_dict replaces variables with a data
for _ in range(1000):
  batch = mnist.train.next_batch(100)
  #print(batch[0][0])
  train_step.run(feed_dict={x: batch[0], y_: batch[1]})

## Evaluate the model

In [26]:
# First we'll figure out where we predicted the correct label. 
# tf.argmax is an extremely useful function which gives you the index of the highest 
# entry in a tensor along some axis. For example, tf.argmax(y,1) is the label our model 
# thinks is most likely for each input, while tf.argmax(y_,1) is the true label. 
# We can use tf.equal to check if our prediction matches the truth.
correct_prediction = tf.equal(tf.argmax(y,1), tf.argmax(y_,1))


In [27]:
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

print(accuracy.eval(feed_dict={x: mnist.test.images, y_: mnist.test.labels}))


0.9267


In [ ]:
#https://www.youtube.com/watch?v=BhpvH5DuVu8

## Build a Multilayer Convolutional Network

In [13]:
#define functions

def weight_variable(shape):
  initial = tf.truncated_normal(shape, stddev=0.1)
  return tf.Variable(initial)

def bias_variable(shape):
  initial = tf.constant(0.1, shape=shape)
  return tf.Variable(initial)


def conv2d(x, W):
  return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')

def max_pool_2x2(x):
  return tf.nn.max_pool(x, ksize=[1, 2, 2, 1],
                        strides=[1, 2, 2, 1], padding='SAME')

In [16]:
# First Convolutional Layer

W_conv1 = weight_variable([5, 5, 1, 32])
b_conv1 = bias_variable([32])

x_image = tf.reshape(x, [-1, 28, 28, 1])

h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1) + b_conv1)
h_pool1 = max_pool_2x2(h_conv1)

In [17]:
# Second Convolutional Layer

W_conv2 = weight_variable([5, 5, 32, 64])
b_conv2 = bias_variable([64])

h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)
h_pool2 = max_pool_2x2(h_conv2)

In [18]:
W_fc1 = weight_variable([7 * 7 * 64, 1024])
b_fc1 = bias_variable([1024])

h_pool2_flat = tf.reshape(h_pool2, [-1, 7*7*64])
h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)

In [19]:
keep_prob = tf.placeholder(tf.float32)
h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)

In [20]:
W_fc2 = weight_variable([1024, 10])
b_fc2 = bias_variable([10])

y_conv = tf.matmul(h_fc1_drop, W_fc2) + b_fc2

In [ ]:
cross_entropy = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=y_, logits=y_conv))
train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)
correct_prediction = tf.equal(tf.argmax(y_conv, 1), tf.argmax(y_, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

with tf.Session() as sess:
  sess.run(tf.global_variables_initializer())
  for i in range(20000):
    batch = mnist.train.next_batch(50)
    if i % 100 == 0:
      train_accuracy = accuracy.eval(feed_dict={
          x: batch[0], y_: batch[1], keep_prob: 1.0})
      print('step %d, training accuracy %g' % (i, train_accuracy))
    train_step.run(feed_dict={x: batch[0], y_: batch[1], keep_prob: 0.5})

  print('test accuracy %g' % accuracy.eval(feed_dict={
      x: mnist.test.images, y_: mnist.test.labels, keep_prob: 1.0}))

step 0, training accuracy 0.06
step 100, training accuracy 0.78
step 200, training accuracy 0.9
step 300, training accuracy 0.98
step 400, training accuracy 0.96
step 500, training accuracy 0.94
step 600, training accuracy 0.94
step 700, training accuracy 0.9
step 800, training accuracy 0.94
step 900, training accuracy 0.9
step 1000, training accuracy 0.96
step 1100, training accuracy 0.98
step 1200, training accuracy 0.96
step 1300, training accuracy 0.9
step 1400, training accuracy 0.98
step 1500, training accuracy 0.96
step 1600, training accuracy 0.94
step 1700, training accuracy 0.96
step 1800, training accuracy 1
step 1900, training accuracy 1
step 2000, training accuracy 1
step 2100, training accuracy 1
step 2200, training accuracy 0.98
step 2300, training accuracy 1
step 2400, training accuracy 0.98
step 2500, training accuracy 0.98
step 2600, training accuracy 1
step 2700, training accuracy 1
step 2800, training accuracy 0.98
step 2900, training accuracy 0.98
step 3000, traini

## Another Multilayer network - 3 layers organized in functions
from  https://www.youtube.com/watch?v=BhpvH5DuVu8

In [7]:
# input > weight > hiddenlayer 1(activation function) > 
#         weights > hiddenlayer 2(activation function) > 
#         weights > hiddenlayer 3(activation function) > 
#         weights > output layer( softmax )
# cost,optimizer(adam,sgd) - back prop
# cycle unitl cost goes resonably down 

n_nodes_hl1=500
n_nodes_hl2=500
n_nodes_hl3=500

# pixel 28x28=784
# 10 output states - numbers 1-10
n_classes =10 
x = tf.placeholder(tf.float32, shape=[None, 784])
y = tf.placeholder(tf.float32, shape=[None, 10])

In [6]:
# entire model in a function 
# https://www.youtube.com/watch?v=PwAGxqrXSCs
import tensorflow as tf
def neural_network_model(data):
    hidden_1_layer = {'weights':(tf.Variable(tf.random_normal([784,n_nodes_hl1 ]))),
                       'biases':(tf.Variable(tf.random_normal([n_nodes_hl1]))) } 
    hidden_2_layer = {'weights':(tf.Variable(tf.random_normal([n_nodes_hl1,n_nodes_hl2 ]))),
                       'biases':(tf.Variable(tf.random_normal([n_nodes_hl2]))) } 
    hidden_3_layer = {'weights':(tf.Variable(tf.random_normal([n_nodes_hl2,n_nodes_hl3 ]))),
                       'biases':(tf.Variable(tf.random_normal([n_nodes_hl3]))) } 
    output_layer = {'weights':(tf.Variable(tf.random_normal([n_nodes_hl3,n_classes ]))),
                       'biases':(tf.Variable(tf.random_normal( [n_classes] ))) } 
                                 
    #Bulding 3 layer model - computentional graph
    # input data in data
    l1=tf.add(tf.matmul(data,hidden_1_layer['weights']), hidden_1_layer['biases'])
    l1=tf.nn.relu(l1)

    l2=tf.add(tf.matmul(l1,hidden_2_layer['weights']), hidden_2_layer['biases'])
    l2=tf.nn.relu(l2)

    l3=tf.add(tf.matmul(l2,hidden_3_layer['weights']),hidden_3_layer['biases'])
    l3=tf.nn.relu(l3)

    output=tf.matmul(l3,output_layer['weights'] + output_layer['biases'])
    return output

In [7]:
# training in a function 
# https://www.youtube.com/watch?v=PwAGxqrXSCs

def train_neural_network(x):
    prediction = neural_network_model(x)  #prediction is an output from the model
    cost=tf.reduce_mean( tf.nn.softmax_cross_entropy_with_logits(logits=prediction,labels=y))
    optimizer = tf.train.AdamOptimizer().minimize(cost)   #with a default learning rate 0.001
    
    hm_epochs = [10]  
    batch_size = 100
    
    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())
        for epoch in hm_epochs:
            epoch_loss = 0
            for _ in range(int(mnist.train.num_examples/batch_size)):
                epoch_x,epoch_y = mnist.train.next_batch(batch_size)
                #here is the action
                _,c=sess.run([optimizer,cost],feed_dict = {x:epoch_x, y:epoch_y})
                epoch_loss+=c
            print('Epoch',epoch,'completed out of ',hm_epochs,'loss',epoch_loss)

        correct= tf.equal(tf.argmax(prediction,1),tf.argmax(y,1))
        accuracy=tf.reduce_mean(tf.cast(correct,'float'))
        print('Accuracy:',accuracy.eval({x:mnist.test.images,y:mnist.test.labels}))
    
train_neural_network(x)           
        

Epoch 10 completed out of  [10] loss 10021665.2293
Accuracy: 0.8543


## 3 layer convolution network organized in functions

In [12]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

n_classes =10 
batch_size =128 
x = tf.placeholder(tf.float32, shape=[None, 784])
y = tf.placeholder(tf.float32, shape=[None, 10])

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [19]:
# entire model in a function 
#https://www.youtube.com/watch?v=mynJtLhhcXk
# https://pythonprogramming.net/cnn-tensorflow-convolutional-nerual-network-machine-learning-tutorial/?completed=/convolutional-neural-network-cnn-machine-learning-tutorial/
import tensorflow as tf

def conv2d(x, W):
  return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')

def maxpool2d(x):
  return tf.nn.max_pool(x, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')


def convolutional_neural_network(x):#, keep_rate):
    weights = {
        # 5 x 5 convolution, 1 input image, 32 outputs
        'W_conv1': tf.Variable(tf.random_normal([5, 5, 1, 32])),
        # 5x5 conv, 32 inputs, 64 outputs 
        'W_conv2': tf.Variable(tf.random_normal([5, 5, 32, 64])),
        # fully connected, 7*7*64 inputs, 1024 outputs
        'W_fc': tf.Variable(tf.random_normal([7*7*64, 1024])),
        # 1024 inputs, 10 outputs (class prediction)
        'out': tf.Variable(tf.random_normal([1024, n_classes]))
    }

    biases = {
        'b_conv1': tf.Variable(tf.random_normal([32])),
        'b_conv2': tf.Variable(tf.random_normal([64])),
        'b_fc': tf.Variable(tf.random_normal([1024])),
        'out': tf.Variable(tf.random_normal([n_classes]))
    }
    
    # Reshape input to a 4D tensor 
    x = tf.reshape(x, shape=[-1, 28, 28, 1])
    # Convolution Layer, using our function
    conv1 = tf.nn.relu(conv2d(x, weights['W_conv1']) + biases['b_conv1'])
    # Max Pooling (down-sampling)
    conv1 = maxpool2d(conv1)
    # Convolution Layer
    conv2 = tf.nn.relu(conv2d(conv1, weights['W_conv2']) + biases['b_conv2'])
    # Max Pooling (down-sampling)
    conv2 = maxpool2d(conv2)
    
      # Fully connected layer
    # Reshape conv2 output to fit fully connected layer
    fc = tf.reshape(conv2, [-1, 7*7*64])
    fc = tf.nn.relu(tf.matmul(fc, weights['W_fc']) + biases['b_fc'])
    
    output = tf.matmul(fc, weights['out']) + biases['out']
    return output
    
    
    

In [20]:
## train convolution NN
def train_neural_network(x):
    prediction = convolutional_neural_network(x)  #prediction is an output from the model
    cost=tf.reduce_mean( tf.nn.softmax_cross_entropy_with_logits(logits=prediction,labels=y))
    optimizer = tf.train.AdamOptimizer().minimize(cost)   #with a default learning rate 0.001
    
    hm_epochs = [10]  
    batch_size = 100
    
    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())
        for epoch in hm_epochs:
            epoch_loss = 0
            for _ in range(int(mnist.train.num_examples/batch_size)):
                epoch_x,epoch_y = mnist.train.next_batch(batch_size)
                #here is the action
                _,c=sess.run([optimizer,cost],feed_dict = {x:epoch_x, y:epoch_y})
                epoch_loss+=c
            print('Epoch',epoch,'completed out of ',hm_epochs,'loss',epoch_loss)

        correct= tf.equal(tf.argmax(prediction,1),tf.argmax(y,1))
        accuracy=tf.reduce_mean(tf.cast(correct,'float'))
        print('Accuracy:',accuracy.eval({x:mnist.test.images,y:mnist.test.labels}))
    
train_neural_network(x)    

Epoch 10 completed out of  [10] loss 1977360.98895
Accuracy: 0.9305
